# Loading and enriching data

In [1]:
import pandas as pd
import requests
import time

In [2]:
df = pd.read_csv('./resources/NY-House-Dataset.csv')

In [6]:
df.head()

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.0,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,New York,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,New York,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766393,-73.980991
2,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,New York,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109
3,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.0,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,New York,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.0,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,New York,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767224,-73.969856


In [ ]:
results = []

# Your OpenCage API key
api_key = 'b8568cb9afc64fad861a69edbddb2658'  # Replace with your actual API key

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    lat = row['LATITUDE']  # Replace with your latitude column name
    lon = row['LONGITUDE']  # Replace with your longitude column name

    # Make the API call
    #https://api.geoapify.com/v1/geocode/reverse?lat=40.618103&lon=-74.0369047&format=json&apiKey=b8568cb9afc64fad861a69edbddb2658
    url = f'https://api.geoapify.com/v1/geocode/reverse?lat={lat}&lon={lon}&format=json&apiKey={api_key}'
   
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        print(data)
        if data['results']:
            # Extract relevant information
            suburb = data['results'][0]['suburb'] if 'suburb' in data['results'][0] else None
            postcode = data['results'][0]['postcode']
            category = data['results'][0]['category'] if 'category' in data['results'][0] else None
            result_type = data['results'][0]['result_type'] if 'result_type' in data['results'][0] else None
            state = data['results'][0]['state']
            city = data['results'][0]['city']

            additional_data = {
                'lat': lat,
                'lon': lon,
                'suburb': suburb,
                'postcode': postcode,
                'category': category,
                'result_type': result_type,
                'state': state,
                'city': city,
            }
            results.append(additional_data)
        else:
            print(f"No results for lat: {lat}, lon: {lon}")
    else:
        print(f"Error: {response.status_code} for lat: {lat}, lon: {lon}")

    # Sleep to avoid hitting API rate limits
    # time.sleep(1)  # Adjust based on API limits

# Create a DataFrame from results
results_df = pd.DataFrame(results)

# Save to a new CSV file
results_df.to_csv('./resources/housing_additional_data_file.csv', index=False)  # Replace with your desired output file name


{'results': [{'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright'}, 'name': 'Astor Court', 'country': 'United States', 'country_code': 'us', 'state': 'New York', 'county': 'New York County', 'city': 'New York', 'postcode': '10022', 'district': 'Manhattan Community Board 5', 'suburb': 'Manhattan', 'street': 'East 55th Street', 'housenumber': '2', 'lon': -73.9744256, 'lat': 40.761318, 'state_code': 'NY', 'distance': 8.530679660519068, 'result_type': 'amenity', 'formatted': 'Astor Court, 2 East 55th Street, New York, NY 10022, United States of America', 'address_line1': 'Astor Court', 'address_line2': '2 East 55th Street, New York, NY 10022, United States of America', 'category': 'catering.restaurant', 'timezone': {'name': 'America/New_York', 'offset_STD': '-05:00', 'offset_STD_seconds': -18000, 'offset_DST': '-04:00', 'offset_DST_seconds': -14400, 'abbreviation_ST

KeyError: 'category'

In [ ]:
for i in range(1, 400):
        response = requests.get('https://api.themoviedb.org/3/\
    movie/top_rated?api_key=aaa7de53dcab3a19afed86880\
    f364e54&language=en-US&page={}'.format(i))
        temp_df = pd.DataFrame(response.json()['results'])[['id',
                    'title', 'overview', 'release_date', 'popularity',
                       'vote_average', 'vote_count']]
        df = df.append(temp_df, ignore_index=True)
else:
    print('Error', response.status_code)